In [2]:
from transformers import AutoModel, AutoTokenizer
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
import matplotlib.pyplot as plt
from torch.cuda.amp import autocast, GradScaler


/home/naver/anaconda3/envs/agent/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
TEST_SIZE = 0.2
DROP_OUT_P = 0.1
num_epochs = 100
checkpoint = "codesage/codesage-small"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device: ", device)

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(checkpoint, trust_remote_code=True, add_eos_token=True)
model = AutoModel.from_pretrained(checkpoint, trust_remote_code=True).to(device)

Using device:  cuda


In [7]:
df = pd.read_csv("/home/naver/Individual/Extened/CodeSage/data.csv")
train_data, valid_data, train_labels, valid_labels = train_test_split(df['code'].values, df['label'].values, test_size=TEST_SIZE, random_state=42)

In [8]:
df.head(5)

,code,label
0,bool virtio_scsi_handle_cmd_req_prepare(VirtIO...,1
1,static void term_forward_char(void)\n\n{\n\n ...,0
2,static int vaapi_build_decoder_config(VAAPIDec...,0
3,void cpu_reset (CPUMIPSState *env)\n{\n mem...,1
4,static struct dirent *local_readdir(FsContext ...,0
